In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification

# Generar datos de ejemplo
n_features = 10
X,y = make_classification(
  n_samples=100, 
  n_features=n_features, 
  n_informative=5, 
  n_classes=3, 
  random_state=42
)

In [ ]:
# Convertir a DataFrame para mejorar visualización
df = pd.DataFrame(X, columns=[f"Feature_{i+1}" for i in range(n_features)])
df['Class'] = y 
display(df.head(3))

### Visualización de Características en Espacio de Búsqueda

Cuando se tiene muchas características (dimensiones), es difícil visualizarlas directamente. Para resolver esto, se usan técnicas como:
- *Reducción de Dimensionalidad*: Proyectan datos $N$-dimensionales a $2/3$-dimensionales.
- *Muestreo de Relaciones*: Crear matrices de correlación o gráficos paralelos.
- *Clustering y Visualización*: Aplicar algoritmos de clustering y visualizar los resultados según categorías o valores.

#### Matriz de Correlación

In [ ]:
plt.figure(figsize=(10, 8))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación entre Características')
plt.tight_layout()
plt.show()

#### Gráfico de Tuplas

In [ ]:
LIMIT = 4
sns.pairplot(df, vars=df.columns[:LIMIT], hue='Class', palette='viridis')
plt.suptitle('Gráfico de Pares de Características', y=1.02)
plt.show()

#### Coordenadas Paralelas

In [ ]:
LIMIT = 5
plt.figure(figsize=(12, 6))
pd.plotting.parallel_coordinates(df, 'Class', cols=df.columns[:LIMIT], colormap='viridis')
plt.title('Coordenadas Paralelas de Características')
plt.xlabel('Características')
plt.ylabel('Valores normalizados')
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.show()

### Aplicación de PCA y Autoencoders para Visualización

PCA y Autoencoders son técnicas de reducción de dimensionalidad: 
- **PCA**: Encuentra las direcciones de máxima varianza (componentes principales).
- **Autoencoders**: Red Neuronal que comprime y reconstruye datos, aprendiendo representaciones compactas.

Ambos permiten visualizar datos complejos en 2/3-dimensionales manteniendo la estructura subyacente.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.manifold import TSNE

X,y = make_classification(
  n_samples=500, 
  n_features=20, 
  n_informative=10, 
  n_classes=4,
  random_state=42
)
# Estandarizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) 

#### PCA

**Objetivo**: Encontrar una *transformación ortogonal* que convierta un conjunto de observaciones de variables posiblemente correlacionadas en un conjunto de valores de variables linealmente no correlacionadas, llamadas *componentes principales*. El primer componente principal tiene la varianza más alta posible, y cada componente sucesivo, a su vez, tiene la más alta alta varianza posible bajo la restricción de que es ortogonal a los componentes anteriores

--- 

1. *Centrar datos*: $\hat{X} = X - \mu$
2. *Matriz de covarianza*: $C = 1/m \hat{X}^T \hat{X}$
3. *Descomposición espectral*: $C = V \Lambda V^T$
4. *Proyección*: $Z = \hat{X} V_k$ donde $V_k$ son los $k$ primeros autovectores 

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"Varianza explicada: {pca.explained_variance_ratio_}")
print(f"Varianza total explicada: {sum(pca.explained_variance_ratio_):.2%}")

# Visualización PCA 2D
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', alpha=0.7, edgecolors='k')
plt.colorbar(scatter, label='Clase')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} varianza)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} varianza)')
plt.title('PCA: Proyección 2D de 20 características')
plt.grid(alpha=0.3)
plt.show()

##### Visualización: PCA 3D

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_scaled)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter3D(X_pca_3d[:, 0], X_pca_3d[:, 1], X_pca_3d[:, 2], c=y, cmap='viridis', alpha=0.7, s=50)
plt.colorbar(scatter, label='Class')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('PCA 3D - Varianza: ' + f'{sum(pca_3d.explained_variance_ratio_):.1%}')
plt.show()

#### Autoencoders

**Descripción**: Red Neuronal que se entrena para reconstruir su entrada. Está compuesto por dos funciones parametrizadas: 
- **Encoder** (**Codificador**): Una función $f_{\phi}$ que transforma un vector de entrada $\textbf{f} \in \mathbb{R}^d$ en una representación latente $\mathbf{z} \in \mathbb{R}^m$
- **Decoder** (**Decodificador**): Una función $g_\theta$ que mapea el código latente $\mathbf{z}$ de vuelta al espacio original, produciendo una reconstrucción $x' \in \mathbb{R}^d$  

---

Se puede expresar de la siguiente forma:
$$\begin{matrix}
\mathbf{z} = f_{\phi}(\mathbf{x}) \\
\mathbf{x}' = g_{\theta}(\mathbf{z}) = g_{\theta}(f_{\theta}(\mathbf{x}))
\end{matrix}$$
donde:
- $\phi:$ son los parámetros (pesos y sesgos) del codificador  
- $\theta:$ son los parámetros del decodificador

--- 

**Espacio Latente** (**Bottleneck**): La dimensionalidad del código latente $m$ es crucial. Típicamente, se fuerza a que $m < d$ (subespacio), lo que obliga a la red a aprender una comprensión con pérdidas de los datos más informativa. Sin embargo, también existen variantes de autoencoders con $m > d$ que requieren fuertes regularizaciones para ser útiles. 

In [ ]:
# Configuración: 20 -> 10 -> 2 -> 10 -> 20
autoencoder = MLPRegressor(
  # Capa bottleneck de 2 dimensiones
  hidden_layer_sizes=(10, 2, 10),
  activation='relu',
  solver='adam',
  max_iter=1000,
  random_state=42,
  verbose=False
)

# Entrenar autoencoder
autoencoder.fit(X_scaled, X_scaled)

# Extraer la representación intermedia (2D)
# Necesitamos acceder a la capa de 2 dimensiones
# Para simplificar, crearemos un encoder manual:
class SimpleAutoencoder:
  def __init__(self):
    self.encoder = MLPRegressor(
      hidden_layer_sizes=(10, 2),
      activation='relu',
      solver='adam',
      max_iter=1000,
      random_state=42
    )

  def fit(self, X, y=None):
    # Entrenar para aprender representación
    self.encoder.fit(X, X[:, :2])  # Aprender a comprimir a 2D
    return self

  def transform(self, X):
    return self.encoder.predict(X)

# Crear y entrenar autoencoder simplificado
ae = SimpleAutoencoder()
ae.fit(X_scaled)
X_ae = ae.transform(X_scaled)

# Visualización Autoencoder
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_ae[:, 0], X_ae[:, 1], c=y, cmap='plasma', alpha=0.7, edgecolors='k')
plt.colorbar(scatter, label='Clase')
plt.xlabel('Dimensión Latente 1')
plt.ylabel('Dimensión Latente 2')
plt.title('Autoencoder: Representación Latente 2D')
plt.grid(alpha=0.3)
plt.show()

#### Comparación

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# PCA
sc1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', alpha=0.7)
axes[0].set_title(f'PCA (Varianza: {sum(pca.explained_variance_ratio_):.1%})')
axes[0].set_xlabel('Componente 1')
axes[0].set_ylabel('Componente 2')
plt.colorbar(sc1, ax=axes[0])

# Autoencoder
sc2 = axes[1].scatter(X_ae[:, 0], X_ae[:, 1], c=y, cmap='plasma', alpha=0.7)
axes[1].set_title('Autoencoder (2D)')
axes[1].set_xlabel('Dimensión 1')
axes[1].set_ylabel('Dimensión 2')
plt.colorbar(sc2, ax=axes[1])

# t-SNE para comparación (no lineal)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)
sc3 = axes[2].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='coolwarm', alpha=0.7)
axes[2].set_title('t-SNE (Referencia no lineal)')
axes[2].set_xlabel('Dimensión 1')
axes[2].set_ylabel('Dimensión 2')
plt.colorbar(sc3, ax=axes[2])

plt.tight_layout()
plt.show()